# Laboratorio Guidato: Analisi del Livello di Trasporto con Scapy

**Obiettivi:**

*   Comprendere il funzionamento pratico dei protocolli TCP e UDP.
*   Analizzare la struttura dei segmenti TCP e dei datagrammi UDP.
*   Simulare un Three-Way Handshake TCP.
*   Osservare il meccanismo di ritrasmissione in TCP.
*   Confrontare TCP e UDP per scegliere il protocollo più adatto in scenari reali.

**Istruzioni:**

Esegui le celle di questo notebook in ordine, una dopo l'altra. Per eseguire una cella, selezionala e premi `Maiusc + Invio` o clicca sul pulsante "Play" a sinistra della cella. Leggi attentamente le spiegazioni e rispondi ai quesiti proposti.

In [ ]:
# Installiamo la libreria Scapy. L'operazione richiede pochi secondi.
!pip install scapy

In [ ]:
# Importiamo tutte le funzioni di Scapy per averle a disposizione
from scapy.all import *

### Sezione 1: Analisi del Three-Way Handshake TCP

Il **Three-Way Handshake** (o "stretta di mano a tre vie") è la procedura con cui TCP stabilisce una connessione affidabile tra un client e un server. Consiste in tre passaggi:

1.  **SYN**: Il client invia un segmento con il flag `SYN` (Synchronize) attivo per iniziare una connessione.
2.  **SYN-ACK**: Il server risponde con un segmento che ha sia il flag `SYN` che il flag `ACK` (Acknowledgement) attivi.
3.  **ACK**: Il client conferma la ricezione inviando un ultimo segmento con il flag `ACK` attivo.

Ora simuleremo questi tre passaggi usando Scapy per creare i pacchetti da zero.

In [ ]:
# Definiamo gli indirizzi IP e le porte
ip_sorgente = "192.168.1.100"
ip_destinazione = "10.0.0.5"
porta_sorgente = 1025 # Porta effimera (tipicamente > 1023)
porta_destinazione = 80   # Porta del servizio web (HTTP)

# 1. Pacchetto SYN (Client -> Server)
# 'S' sta per SYN. Il numero di sequenza (seq) è scelto casualmente, ma qui lo fissiamo a 100 per semplicità.
pacchetto_syn = IP(src=ip_sorgente, dst=ip_destinazione)/TCP(sport=porta_sorgente, dport=porta_destinazione, flags='S', seq=100)

# 2. Pacchetto SYN-ACK (Server -> Client)
# 'SA' sta per SYN-ACK. Il server risponde con il suo numero di sequenza (es. 500)
# e incrementa di 1 il seq del client nel campo ack, a conferma della ricezione.
pacchetto_syn_ack = IP(src=ip_destinazione, dst=ip_sorgente)/TCP(sport=porta_destinazione, dport=porta_sorgente, flags='SA', seq=500, ack=pacchetto_syn.seq + 1)

# 3. Pacchetto ACK (Client -> Server)
# 'A' sta per ACK. Il client risponde incrementando di 1 il seq del server per confermare la ricezione del SYN-ACK.
pacchetto_ack = IP(src=ip_sorgente, dst=ip_destinazione)/TCP(sport=porta_sorgente, dport=porta_destinazione, flags='A', seq=pacchetto_syn.seq + 1, ack=pacchetto_syn_ack.seq + 1)

print("Pacchetti del Three-Way Handshake creati con successo!")

In [ ]:
# Visualizziamo l'intestazione completa del pacchetto SYN-ACK
# Il comando .show() ci mostra in dettaglio tutti i campi del pacchetto, sia a livello IP che TCP.
print("--- Dettaglio del pacchetto SYN-ACK ---")
pacchetto_syn_ack.show()

### Quesito 1

Osserva attentamente l'output della cella precedente (`pacchetto_syn_ack.show()`). Rispondi alle seguenti domande:

1.  Quali sono i valori (in decimale) delle porte sorgente (`sport`) e destinazione (`dport`)? Corrispondono a quelli che abbiamo definito?
2.  Nel campo `flags`, quali flag sono attivi? Riesci a spiegare il perché?
3.  Qual è il valore del campo `ack`? A cosa corrisponde rispetto al primo pacchetto inviato (`pacchetto_syn`)?

### Sezione 2: Troubleshooting: La Ritrasmissione TCP

TCP è un protocollo **affidabile**. Questo significa che garantisce la consegna dei dati. Se un pacchetto viene perso nella rete, il mittente non riceve l'ACK di conferma entro un certo tempo (chiamato RTO, *Retransmission Time-Out*). Scaduto il timer, il mittente **ritrasmette** lo stesso pacchetto.

Simuliamo questo scenario: un client invia un pacchetto di dati che viene perso. Dopo un po', lo ritrasmette.

In [ ]:
# Assumiamo che la connessione sia già stabilita.
# Il client invia un pacchetto con dei dati (payload).
seq_iniziale_dati = pacchetto_ack.seq
ack_iniziale_dati = pacchetto_ack.ack

# 1. Primo tentativo di invio (pacchetto perso)
pacchetto_dati_1 = IP(src=ip_sorgente, dst=ip_destinazione)/TCP(sport=porta_sorgente, dport=porta_destinazione, flags='PA', seq=seq_iniziale_dati, ack=ack_iniziale_dati)/"Questo e' il payload"

# 2. Scade il timer RTO, il client ritrasmette lo stesso pacchetto
# Nota come il numero di sequenza (seq) sia identico al precedente!
pacchetto_ritrasmesso = IP(src=ip_sorgente, dst=ip_destinazione)/TCP(sport=porta_sorgente, dport=porta_destinazione, flags='PA', seq=seq_iniziale_dati, ack=ack_iniziale_dati)/"Questo e' il payload"

print("--- Pacchetto Dati Originale (perso) ---")
pacchetto_dati_1.show()
print("\n--- Pacchetto Dati Ritrasmesso ---")
pacchetto_ritrasmesso.show()

### Quesito 2

Confrontando i due pacchetti mostrati nell'output qui sopra:

1.  Quale campo, oltre al contenuto del payload, dimostra in modo inequivocabile che il secondo pacchetto è una **ritrasmissione** del primo?
2.  Questo comportamento fa parte di quale meccanismo di controllo di TCP che lo rende un protocollo "affidabile"?

### Sezione 3: Confronto con UDP e Scelta Progettuale

UDP (User Datagram Protocol) è l'altro principale protocollo del Livello di Trasporto. A differenza di TCP, è **non orientato alla connessione** e **non affidabile** (*best-effort*). Questo lo rende molto più "leggero" e veloce, perché non deve gestire numeri di sequenza, acknowledgment o handshake.

Creiamo un pacchetto UDP, ad esempio una richiesta DNS alla porta 53.

In [ ]:
# Definiamo un datagramma UDP per una richiesta DNS
# L'IP di destinazione 8.8.8.8 è un server DNS pubblico di Google.
pacchetto_udp = IP(src=ip_sorgente, dst="8.8.8.8")/UDP(dport=53)/DNS(rd=1, qd=DNSQR(qname="www.google.com"))

print("--- Dettaglio del datagramma UDP ---")
pacchetto_udp.show()

### Quesito 3

Osserva l'header UDP e confrontalo mentalmente con quello TCP visto nella Sezione 1. Rispondi a queste domande:

1.  Elenca almeno due campi fondamentali di TCP (es. `seq`, `ack`, `flags`) che sono **mancanti** nell'header di UDP.
2.  **Scenario Progettuale**: Stai sviluppando un'applicazione per lo **streaming video in tempo reale** (come una diretta su Twitch o una videochiamata). Quale protocollo sceglieresti tra TCP e UDP? Motiva la tua risposta considerando i pro e i contro di ciascuno in questo specifico contesto (pensa a cosa è più importante: che arrivino *tutti* i dati o che arrivino *in fretta*?).

---
## Soluzioni

### Soluzione Quesito 1

1.  **Porte**: La porta sorgente (`sport`) è 80 e la porta destinazione (`dport`) è 1025. I valori sono corretti e **invertiti** rispetto al pacchetto `pacchetto_syn`, perché questo è un pacchetto di risposta (dal server al client).
2.  **Flags**: Sono attivi i flag `SYN` e `ACK` (Scapy li mostra come `SA`). Questo perché il server deve sia confermare (`ACK`) la richiesta di connessione del client, sia inviare la sua proposta di sincronizzazione (`SYN`).
3.  **Valore Ack**: Il campo `ack` ha valore `101`. Questo corrisponde al `seq` del pacchetto precedente (`100`) incrementato di 1, come previsto dal protocollo TCP per confermare la ricezione di quel byte.

### Soluzione Quesito 2

1.  **Campo Chiave**: Il campo che prova la ritrasmissione è il **Numero di Sequenza (`seq`)**. In una comunicazione TCP normale, il `seq` number viene incrementato per ogni nuovo blocco di dati inviato. Trovare due pacchetti distinti con lo stesso `seq` number indica che il secondo è una ritrasmissione del primo.
2.  **Meccanismo**: Questo meccanismo è il **controllo di errore** e la **gestione dell'affidabilità** di TCP. Il protocollo usa i riscontri (ACK) e i timer di ritrasmissione (RTO) per assicurarsi che ogni pacchetto arrivi a destinazione. Se un ACK non arriva in tempo, il segmento viene inviato di nuovo.

### Soluzione Quesito 3

1.  **Campi Mancanti**: UDP non ha i campi `seq` (Sequence Number), `ack` (Acknowledgement Number) e `flags` (SYN, ACK, FIN, etc.). Manca anche il campo `window` per il controllo di flusso.
2.  **Scenario Progettuale**: Per lo streaming video in tempo reale, la scelta corretta è **UDP**. 
    *   **Motivazione**: Nello streaming live, la **velocità** e la **bassa latenza** sono più importanti della completezza dei dati. Se un frame video viene perso, è meglio saltarlo e mostrare il successivo piuttosto che fermare lo streaming per aspettare la ritrasmissione (come farebbe TCP), cosa che causerebbe blocchi e lag (il cosiddetto "buffering"). UDP, essendo più leggero e non gestendo ritrasmissioni, è perfetto per inviare un flusso continuo di dati dove la perdita di un singolo pacchetto non compromette l'esperienza utente in modo critico.